In [2]:
"""
More info about TabNet at:
https://github.com/dreamquark-ai/tabnet
"""
!pip install pytorch-tabnet

In [3]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime
from scipy import stats

In [4]:
#Preprocess the dataset
np.random.seed(0)
cwd = os.getcwd()
dataset_path = Path(cwd)/'Catalogue.csv'
dataset = pd.read_csv(dataset_path, encoding='latin-1')

# Extract month from the date & time
datetimes = list(dataset['Zaman (UTC)'].values)
months = []
for dt in datetimes:
    month = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S').month
    months.append(month)
    
dataset = dataset[['Enlem', 'Boylam', 'Derinlik', 'Büyüklük']]
dataset.insert(2, 'Ay', months, True) # insert month info at index 2 and check if the column name already exists

# Obtain features from Derinlik and Büyüklük
hmean_features = []
avg_features = []
max_features = []
moment_features = []
kurtosis_features = []
gmean_features = []
for index, row in dataset.iterrows():
    obtain_features = [row['Derinlik'], row['Büyüklük']]
    hmean_features.append(stats.hmean(obtain_features)) 
    avg_features.append(stats.tmean(obtain_features))
    max_features.append(stats.tmax(obtain_features))
    moment_features.append(stats.moment(obtain_features, moment=2))
    kurtosis_features.append(stats.kurtosis(obtain_features))
    gmean_features.append(stats.gmean(obtain_features))
dataset['Harmonic Mean'] = hmean_features
dataset['Average'] = avg_features
dataset['Max'] = max_features
dataset['Moment'] = moment_features
#dataset['Kurtosis'] = kurtosis_features
dataset['Geometric Mean'] = gmean_features

# Split the dataset
if 'Set' not in dataset.columns:
    dataset['Set'] = np.random.choice(['train', 'val', 'test'], p=[.7, .1, .2], size=(dataset.shape[0],))
    
train_indices = dataset[dataset.Set == 'train'].index
val_indices = dataset[dataset.Set == 'val'].index
test_indices = dataset[dataset.Set == 'test'].index
#dataset.drop('Set', axis=1, inplace=True)

#print(dataset)

# Check if there are NaN or empty string values
#print(np.where(pd.isnull(dataset)))
#print(np.where(dataset.applymap(lambda x: x == '')))

target = 'Büyüklük'
unused_features = ['Set']
features = [col for col in dataset.columns if col not in unused_features+[target]]

x_train = dataset[features].values[train_indices]
y_train = dataset[target].values[train_indices].reshape(-1,1)

x_val = dataset[features].values[val_indices]
y_val = dataset[target].values[val_indices].reshape(-1,1)

x_test = dataset[features].values[test_indices]
y_test = dataset[target].values[test_indices].reshape(-1,1)

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:330: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


In [5]:
# Train the model
model = TabNetRegressor()
max_epochs = 1000

model.fit(
        X_train=x_train, y_train=y_train,
        eval_set=[(x_train, y_train), (x_val, y_val)],
        eval_name=['train', 'val'],
        #eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
        eval_metric=['rmse'],
        max_epochs=max_epochs,
        patience=50,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        drop_last=False
        )

Device used : cuda
epoch 0  | loss: 12.39476| train_rmse: 15.96333| val_rmse: 15.57166|  0:00:01s
epoch 1  | loss: 1.66102 | train_rmse: 17.79318| val_rmse: 18.22072|  0:00:02s
epoch 2  | loss: 0.48044 | train_rmse: 5.3028  | val_rmse: 5.28434 |  0:00:02s
epoch 3  | loss: 0.27566 | train_rmse: 5.22005 | val_rmse: 5.30728 |  0:00:03s
epoch 4  | loss: 0.23696 | train_rmse: 4.38711 | val_rmse: 4.41623 |  0:00:04s
epoch 5  | loss: 0.17321 | train_rmse: 2.83586 | val_rmse: 2.86329 |  0:00:05s
epoch 6  | loss: 0.14284 | train_rmse: 2.31648 | val_rmse: 2.32136 |  0:00:06s
epoch 7  | loss: 0.12892 | train_rmse: 1.68667 | val_rmse: 1.69696 |  0:00:07s
epoch 8  | loss: 0.11269 | train_rmse: 1.43127 | val_rmse: 1.44887 |  0:00:08s
epoch 9  | loss: 0.09608 | train_rmse: 1.76439 | val_rmse: 1.78202 |  0:00:09s
epoch 10 | loss: 0.08838 | train_rmse: 1.56051 | val_rmse: 1.57949 |  0:00:10s
epoch 11 | loss: 0.07615 | train_rmse: 1.31333 | val_rmse: 1.32711 |  0:00:11s
epoch 12 | loss: 0.07044 | train_

In [6]:
# Test the model
y_pred = model.predict(x_test)
test_score = mean_squared_error(y_pred=y_pred, y_true=y_test)
print(f'Best Valid Score: {model.best_cost}')
print(f'Test Score: {test_score}')

Best Valid Score: 0.05374865114882252
Test Score: 0.0036225906952328285


In [7]:
# Save the model
save_path = './tabnet_model'
saved_path = model.save_model(save_path)

Successfully saved model at ./tabnet_model.zip


In [8]:
# Define a new model and load state dict weights
loaded_model = TabNetRegressor()
loaded_model.load_model(saved_path)

Device used : cuda
Device used : cuda


In [9]:
# Test the loaded model and assert the results are same
pred = loaded_model.predict(x_test)
score = mean_squared_error(pred, y_test)
print(f'Test Score of Loaded Model: {score}')
assert(test_score == score)

Test Score of Loaded Model: 0.0036225906952328285


In [12]:
# Print the actual and predicted values for test samples
for index, y in enumerate(y_test):
    print(f'Index: {index}\tActual: {y}\tPrediction: {pred[index]}')

Index: 0	Actual: [4.1]	Prediction: [4.067148]
Index: 1	Actual: [4.3]	Prediction: [4.258804]
Index: 2	Actual: [4.4]	Prediction: [4.3276477]
Index: 3	Actual: [4.1]	Prediction: [4.0906143]
Index: 4	Actual: [4.1]	Prediction: [4.0730066]
Index: 5	Actual: [4.2]	Prediction: [4.177415]
Index: 6	Actual: [4.8]	Prediction: [4.647723]
Index: 7	Actual: [4.3]	Prediction: [4.3292704]
Index: 8	Actual: [4.]	Prediction: [3.9816704]
Index: 9	Actual: [4.2]	Prediction: [4.1775303]
Index: 10	Actual: [4.8]	Prediction: [4.649836]
Index: 11	Actual: [4.1]	Prediction: [4.136082]
Index: 12	Actual: [4.2]	Prediction: [4.219333]
Index: 13	Actual: [5.3]	Prediction: [5.294671]
Index: 14	Actual: [4.1]	Prediction: [4.062943]
Index: 15	Actual: [4.7]	Prediction: [4.577261]
Index: 16	Actual: [4.2]	Prediction: [4.1553965]
Index: 17	Actual: [4.]	Prediction: [3.977447]
Index: 18	Actual: [4.1]	Prediction: [4.0752506]
Index: 19	Actual: [4.2]	Prediction: [4.178102]
Index: 20	Actual: [4.1]	Prediction: [4.0685472]
Index: 21	Actual